In [ ]:
# the following code was written by: Author: Ewelina Dobrowolska - ewelina.dobrowolska@serco.com
# lightly edited by Lilly Jones to evaluate for SCE project

#MODULE NAME                                             #DESCRIPTION
import os                                                #data access to file manager 
import matplotlib as mpl                                 #create visualizations
import matplotlib.colors as colors                       #create visualizations
import matplotlib.image as mpimg                         #create visualizations
import matplotlib.pyplot as plt                          #create visualizations
import matplotlib.cm as cm                               #create visualizations
import skimage.exposure as exposure                      #collection of algorithms for image processing
import pandas as pd                                      #data analysis and manipulation
import numpy as np                                       #scientific computing
import subprocess                                        #allows you to spawn new processes
import snappy                                            #SNAP Python interface
import jpy                                               #Python-Java bridge
import imageio                                           #visualize and process images 
import rasterio as rio                                   #working with geospatial raster data
import rioxarray as rxr                                  #geospatial xarray extension employed in rasterio
import rasterstats as rs                                 #summarizing geospatial raster datasets based on vector geometries
import gdal                                              #Geospatial Data Abstraction Library for manipulating vecotr and raster data
import rasterio.plot                                     #imagrs visualization with rasterio 
from glob import iglob                                   #data access in file manager
from snappy import ProductIO                             #reading product in SNAP-Python interface
from snappy import GPF                                   #allows to create products in SNAP-Python
from snappy import Product                               #reading product in SNAP-Python interface
from snappy import ProductData                           #access to product data in SNAP-Python
from snappy import ProductUtils                          #access specific functions of snappy
from snappy import FlagCoding                            #writing data to the band flag coding
from snappy import WKTReader                             #reading the geometry from WKT coordinates
from zipfile import ZipFile                              #allows to read .zip files       
from osgeo import gdal, gdal_array, ogr                  #import major modules from gdal library 
import geopandas as gpd                                  #extention of databases used in pandas to allow spatial operations 
from pyspatialml import Raster                           #applying scikit-learn machine learning models to 'stacks' of raster datasets
import seaborn as sns                                    #data visualization library based on matplotlib
from rasterio.plot import show                           #visualization of images using rasterio plot  
from pathlib import Path                                 #read raster product path 
%matplotlib inline          

#Change module setting - the maximum width of column in dataframe displayed
pd.options.display.max_colwidth = 80                     #longer text in pd data frame (df)
print('All modules loaded')                              #inform user once packages are loaded



In [ ]:
#Load aux data
#Directory containing downloaded Sentinel 2 image (unzipped file)
input_directory = '/shared/Training/PY02_ForestBiomass_Sentinel2/Original/'
#Directory where all output products will be saved 
output_directory = '/shared/Training/PY02_ForestBiomass_Sentinel2/Processing/'
#Path to the input product (unzipped because of the atmospheric correction which requires data in .xml format)
SL1C_product_path = input_directory + 'S2A_MSIL1C_20170119T074231_N0204_R092_T37NEH_20170119T075734.SAFE/MTD_MSIL1C.xml'
#path to the training data points previously created
points_path = '/shared/Training/PY02_ForestBiomass_Sentinel2/AuxData/points_agb_2017.shp'

In [ ]:
#Function to create band composites using snappy-Python environment 
#bands_com - are the bands which will be used for the creation of bands composites
#filename - the path to the file with its name where the output will be stored
# format_im - image format used to stored the results 
def write_band_composites(bands_com, filename, format_im):
    image_info = ProductUtils.createImageInfo(bands_com, True, ProgressMonitor.NULL)
    im = ImageManager.getInstance().createColoredBandImage(bands_com, image_info, 0)
    JAI.create("filestore", im, filename, format_im)

#Function to calculate vegetation indices from band combinations
#raster_output_name - name of the vegetation index which will be saved
# VI_name - name of the vegetation index 
# cmap_output_name - name of the png file where the file will be stored with the colormap applied 
#plot_title - tile of the image produced 
def create_vegetation_indices(raster_output_name, VI_name, cmap_output_name, plot_title):
    np.seterr(divide='ignore', invalid='ignore')
    kwargs = src.meta
    kwargs.update(
        dtype=rasterio.float32,
        count = 1)
    #create the Vegetation Index geotiff file with the colormap 
    with rasterio.open(VI_output + raster_output_name, 'w', **kwargs) as dst:
                        dst.write_band(1, VI_name.astype(rasterio.float32))
    plt.imsave(VI_output+cmap_output_name, VI_name, cmap=plt.cm.RdYlGn)
    plt.imshow(VI_name, cmap = cm.RdYlGn) 
    plt.colorbar() 
    plt.title(plot_title) 
    plt.show()
#Normalizes numpy arrays into scale -1.0 - 1.0
#array - indicates in this case the array of the image to normalize 
def normalize(array): 
    array_min, array_max = array.min(-1), array.max(1)
    return ((array - array_min)/(array_max - array_min))
#Extract vegetation indices values based on the polygons buffers (zones) previously created 
#VI_data - indicate the vegetation index transformed into array 
#poly_buffer_path - path to the file with the 20 m buffer stored in the local folder 
#VI_mean - vegetation index mean extracted from the polygons 
def zonal_statistics(VI_data):
    VI_mean = rs.zonal_stats(poly_buffer_path,
                             VI_data.values,
                             nodata=-999,
                             affine=VI_data.rio.transform(),
                             geojson_out=True,
                             copy_properties=True,
                             stats="mean")
    return VI_mean
#create geodataframe with the vegetation indices values 
#VI_mean - vegetation index mean value extracted to place int the geodataframe
#VI_gdf - geodataframe with the vegetation index values 
def create_GeoDataFrame(VI_mean):
    VI_gdf = gpd.GeoDataFrame.from_features(VI_mean)
    return VI_gdf
#create dataframe with the vegetation indcices without geo information 
#VI_df - dataframe which will be created with the vegetation index for each index
def create_DataFrame(VI_gdf):
    VI_df = pd.DataFrame(VI_gdf.drop(columns='geometry'))
    VI_df = VI_df.drop(columns=['OID_', 'POINT_X', 'POINT_Y'])
    return VI_df
# Create a GeoTIFF file with the given data (in this case this will be used in the step where we prepare new raster with predicted biomass values
#outRaster -indicates the output raster created
#data - predictor variables (vegetation indices - which will predict the AGB values)
#to create the raster gdal library will be used 
#projection - projection of the raster set 
def createGeotiff(outRaster, data, geo_transform, projection):
    driver = gdal.GetDriverByName('GTiff')
    rows, cols = data.shape
    rasterDS = driver.Create(outRaster, cols, rows, 1, gdal.GDT_Float32)
    rasterDS.SetGeoTransform(geo_transform)
    rasterDS.SetProjection(projection)
    band = rasterDS.GetRasterBand(1)
    band.WriteArray(data)
    rasterDS = None

In [ ]:
#the following requires the Sen2cor toolbox

#Read input Sentinel 2 L1C image to snappy 
sentinel_L1C = snappy.ProductIO.readProduct(SL1C_product_path)
#Atmospheric correction from 1c to 2A level using Sen2Cor toolbox version 280
parameters = snappy.HashMap() #setting parameters for processing the image 
parameters.put('resolution', 'ALL')  #final product will have all three resolutions: 10, 20 and 60m
s2A = snappy.GPF.createProduct('Sen2Cor280', parameters, sentinel_L1C) #run processing and save the product
print('Done.') #once processing is finished user will be informed about the end of processing 

In [ ]:
# resampling, band subset, band composition

#Declare the path to the pre-processed S2A image
s2A_path = input_directory + 'S2A_MSIL2A_20170119T074231_N9999_R092_T37NEH_20210805T145407.SAFE/MTD_MSIL2A.xml'
s2A_read = ProductIO.readProduct(s2A_path) #read product using snappy 
name_original = s2A_read.getName() #substract the name of the product to write in the output name 
#Set parameters for resampling:
parameters = snappy.HashMap()
parameters.put('targetResolution', 20) #reference band resolution 20 m 
resample = snappy.GPF.createProduct('Resample', parameters, s2A_read) #perform resampling of the image bands
#Show comunication when the processing is finished
print('Resampled')
#Set parameters for subset of the resampled product
HashMap = snappy.jpy.get_type('java.util.HashMap')
parameters = HashMap()
parameters.put('copyMetadata', True)
parameters.put('sourceBands', 'B2,B3,B4,B5,B6,B8,B11,B12') #selection of bands to store in the final output product (subset)
parameters.put('region','0,0,5490,5490') #set height and weight of the raster band for all the bands 
subset = snappy.GPF.createProduct('Subset', parameters, resample) #perform subset using as input file resampled product
print('Subset created') #comunicate when the subset is being created 
subset_path = output_directory + 'S2/' #path where the subset product will be stored
ProductIO.writeProduct(subset,subset_path + name_original + '_subset.dim', 'BEAM-DIMAP') #saving subset as .dim format
print('Product written')
ProductIO.writeProduct(subset,subset_path +name_original + '_subset.tif', 'GeoTIFF')     #saving subset as .tiff format
print('GeoTIFF product written')

#Indicate the path to atmospherically corrected Sentinel-2A image: 
product_path = output_directory + 'S2/S2A_MSIL2A_20170119T074231_N9999_R092_T37NEH_20210805T145407_subset.tif'
subset_product = ProductIO.readProduct(product_path) #read image in snappy module 
#first we need to import all the bands and give them the shorter name to display and to read properly in snappy module 
B2 = subset_product.getBand('B2')    #blue band
B3 = subset_product.getBand('B3')    #green band
B4 = subset_product.getBand('B4')    #red band (visible spectrum)
B8 = subset_product.getBand('B8')    #NIR
B11 = subset_product.getBand('B11')  #SWIR1
B12 = subset_product.getBand('B12')  #SWIR2

#Using snappy to create band compositions (import necessary modules)
from snappy import (ProductIO, ProductUtils, ProgressMonitor)
path_band_composites = '/shared/Training/PY02_ForestBiomass_Sentinel2/Processing/RGB/' #path to output product 
image_format = 'PNG' #band composites will be saved to .png image format 
# Java type definitions required for image generation in snappy
jpy_s = snappy.jpy                      
Color = jpy_s.get_type('java.awt.Color')
ColorPoint = jpy_s.get_type('org.esa.snap.core.datamodel.ColorPaletteDef$Point')  
ColorPaletteDef = jpy_s.get_type('org.esa.snap.core.datamodel.ColorPaletteDef')  
ImageInfo = jpy_s.get_type('org.esa.snap.core.datamodel.ImageInfo')            
ImageLegend = jpy_s.get_type('org.esa.snap.core.datamodel.ImageLegend')          
ImageManager = jpy_s.get_type('org.esa.snap.core.image.ImageManager')          
JAI = jpy_s.get_type('javax.media.jai.JAI')                                    
RenderedImage = jpy_s.get_type('java.awt.image.RenderedImage')
# Disable JAI native MediaLib extensions - print true if the extensons are ready
System = jpy_s.get_type('java.lang.System')
System.setProperty('com.sun.media.jai.disableMediaLib', 'true')

#Create band composites using pre-defined funcnction from section 3 
write_band_composites ([B4, B3, B2], path_band_composites + 'RGB_20170119.png', image_format) #RGB band composites - Natural color 
write_band_composites ([B8, B4, B3], path_band_composites + 'fColor_20170119.png', image_format) # False-color band composite 
write_band_composites ([B8, B11, B2], path_band_composites + 'vegfColor_20170119.png', image_format) #False color band composite 2 
print('Band composites written') #inform user when the band composites are saved in output folder 

# Create Gif from the list of .png files
from pathlib import Path
image_path = Path(path_band_composites) #path to folder with band composites 
images = list(image_path.glob('*.png')) #list all products using glob module with the extension .png
image_list = []
for file_name in images:
    image_list.append(imageio.imread(file_name))
#Create GIF from the list of band composites 
gif = imageio.mimwrite('/shared/Training/PY02_ForestBiomass_Sentinel2/Processing/RGB/'+'RGB_composite.gif', image_list, fps=2)
print('RGB bands composites saved as Giff')



In [ ]:
# vegetation indices

print(list(subset_product.getBandNames())) #print names of bands

# Load Sentinel 2 bands needed for calculation of Vegetation indices and masking no data values in case there are some before the calculations
with rasterio.open(product_path) as src:
    band_4 = src.read(3, masked=True) #red band
    band_5= src.read(4, masked=True) #VRed-Edge band   
    band_8= src.read(6, masked=True) #NIR band
    band_11= src.read(7, masked=True) #SWIR1 band
np.seterr(divide='ignore', invalid='ignore')
#set output path to the folder where the Vegetation indices will be stored 
VI_output = '/shared/Training/PY02_ForestBiomass_Sentinel2/Processing/VI_output/' 

In [ ]:
#NDVI

#Equation to calculate NDVI index
ndvi = (band_8.astype(float) - band_4.astype(float)) / (band_8 + band_4)
#create NDVI bands with the use of "create_vegetation_indices function from section 3 - User-defined functions"
#the results will be written into tif and png file with the colormap applied
ndvi_results = create_vegetation_indices('1_NDVI.tif', ndvi, '1_NDVI_cmap.png', 'NDVI_results')

In [ ]:
# normalized difference index

# Calculate NDI45 using formula above
ndi45 = (band_5.astype(float) - band_4.astype(float)) / (band_5 + band_4)
#create NDI45 band with the use of "create_vegetation_indices function from section 3 - User-defined functions"
ndi45_results = create_vegetation_indices('2_NDI45.tif', ndi45, '2_NDI45_cmap.png', 'NDI45_results')

In [ ]:
# soil adjusted vegetation index

#Calculate SAVI using formula above
l = 0.428 # L parameter assigned
savi = (band_8.astype(float) - band_4.astype(float)) / (band_8 + band_4 + l) * (1+l)
#normalize raster results to the scale: -1 tp 1:
savi_n = normalize(savi)
#create SAVI band with the use of "create_vegetation_indices function from section 3 - User-defined functions"
savi_results = create_vegetation_indices('3_SAVI.tif', savi_n, '3_SAVI_cmap.png', 'SAVI_results')

In [ ]:
# normalized difference moisture index

#Calculate NDMI using formula above
ndmi = (band_8.astype(float) - band_11.astype(float)) / (band_8 +  band_11 ) 
#create NDMI band with the use of "create_vegetation_indices function from section 3 - User-defined functions"
ndmi_results = create_vegetation_indices('4_NDMI.tif', ndmi, '4_NDMI_cmap.png', 'NDMI_results')

In [ ]:
# raster stack of veg indices

#import list of files (rasters) to create image stack 
from glob import glob
glist = sorted(glob("/shared/Training/PY02_ForestBiomass_Sentinel2/Processing/VI_output/*.tif")) 
#import list of products with the extension .tif 
glist #print list of products which will be included in raster stack

#read all metadata of all single raster bands: 
with rasterio.open(glist[0]) as src0:
    meta = src0.meta
meta.update(count = len(glist))
#rad each single band of vegetation indices and load them to rasterio module
with rasterio.open('/shared/Training/PY02_ForestBiomass_Sentinel2/Processing/VIs_stack.tif', 'w', **meta) as dst:
    for id, layer in enumerate(glist, start=1):
        with rasterio.open(layer) as src1:
            dst.write_band(id, src1.read(1))
#save raster stack in the output folder
with rio.open('/shared/Training/PY02_ForestBiomass_Sentinel2/Processing/VIs_stack.tif') as stack_src:
    stack_data = stack_src.read(masked=True)
    stack_meta = stack_src.profile
stack_meta #display metadata of the produt 

In [ ]:
# import training points shapefile

#READ SHAPEFILE INTO PYTHON ENVIRONMENT
pts = gpd.read_file(points_path) #points will be read into Python using geopandas module 
print(pts.count()) #here we will display basic information about the points shapefile 

#IMPORT VEGETATION RASTER STACK USING RASTERIO PACKAGE 
VIs_stack = rasterio.open('/shared/Training/PY02_ForestBiomass_Sentinel2/Processing/VIs_stack.tif') #set path to the raster stack 
fig, ax = plt.subplots(figsize=(10,10)) #set the size of the plot 
pts.plot(ax=ax, color = 'orangered') #plot imported points and set their color 
show(VIs_stack, ax=ax) #display the image (raster stack) together with points 

In [ ]:
# create polygon shapefile with 20 m buffer

#CREATE A BUFFER AROUND TRAINING POINTS  
poly = pts.copy()  #make a copy of points shapefile previously loaded
#Buffer each point using a 20 meter circle radius and replace the point geometry with the new buffered geometry 
poly["geometry"] = pts.geometry.buffer(20) 
poly.head() 
# Export the buffered point layer as a shapefile to use in zonal stats 
poly_buffer_path = ("poly_agb_buffer_20m.shp") 
poly.to_file(poly_buffer_path)

In [ ]:
#Perform zonal statistics
#First we will open each file with vegetation index representation as an xarray and save it to VIs_data variable 
#Secondly we will calculate the mean of each VIs pixel value which fell inside the polygon created before. 
# We will save the results of statistics to new variable VIs_mean

NDVI_data = rxr.open_rasterio(VI_output + '1_NDVI.tif', masked=True).squeeze()
NDI45_data = rxr.open_rasterio(VI_output + '2_NDI45.tif', masked=True).squeeze()
SAVI_data = rxr.open_rasterio(VI_output + '3_SAVI.tif', masked=True).squeeze()
NDMI_data = rxr.open_rasterio(VI_output + '4_NDMI.tif', masked=True).squeeze()
NDVI_mean = zonal_statistics(NDVI_data)
NDI45_mean = zonal_statistics(NDI45_data)
NDMI_mean = zonal_statistics(NDMI_data)
SAVI_mean = zonal_statistics(SAVI_data)

In [ ]:
#create geodataframes and dataframes with the Vegetation Indices values. 
# Function to create dataframe is saved in the section 3 - User-defined functions

gdf1 = create_GeoDataFrame(NDVI_mean)
df1= create_DataFrame(gdf1)
df1.rename(columns = {'_AGB_2017m': 'AGB_2017','mean':'NDVI'}, inplace = True)
gdf2 = create_GeoDataFrame(NDI45_mean)
df2= create_DataFrame(gdf2)
df2.rename(columns = {'mean':'NDI45'}, inplace = True)
gdf3 = create_GeoDataFrame(SAVI_mean)
df3= create_DataFrame(gdf3)
df3.rename(columns = {'mean':'SAVI'}, inplace = True)
gdf4 = create_GeoDataFrame(NDMI_mean)
df4= create_DataFrame(gdf4)
df4.rename(columns = {'mean':'NDMI'}, inplace = True)

#CREATE UNIQUE DATAFRAME WHICH CONTAINS ALL THE COLUMNS WITH VIS MEAN VALUES AT EACH POINT LOCATION 
training_dataset = pd.concat([df1,df2, df3, df4], axis=1, join='inner') #combine four dataframes previously created into unique database
training_dataset = training_dataset.drop(columns=['ID', '_AGB_2017m']) #remove duplicated colums from the final databse (AGB_2017m and ID)

# WRITE RECORDS OF THE TRAINING DATABASE TO .CSV FILE
training_dataset.to_csv('training_dataset.csv', header =True, index=False)
#display 6 first rows of the database created 
training_dataset.head()

In [ ]:
#Visual representation of the correlation matrix using seaborn Python module
corrMatrix = training_dataset.corr() #create first simple correlation matrix 
sns.heatmap(corrMatrix, annot=True) #plot the correlation matix as a heatmap
plt.show()

In [ ]:
# linear relationship between VIs and AGB values

#Importing dataset using pandas package
df = pd.read_csv('training_dataset.csv') 

#scatterplots between all four variables and the AGB values derived from the reference dataset 
#we will plot each vegetation index against the AGB values in order to see the relationship between these values and the 
#character of this relation. 

df.plot.scatter(x='NDVI', y='AGB_2017', s=60,c='green')
df.plot.scatter(x='NDI45', y='AGB_2017', s=60,c='blue')
df.plot.scatter(x='SAVI', y='AGB_2017', s=60,c='red')
df.plot.scatter(x='NDMI', y='AGB_2017', s=60,c='orange')

In [ ]:
# multiple linear regression model

#CHOOSE COLUMNS WHICH INCLUDE ALL VARIABLES (X- PREDICTORS - VIs AND y - DEPENDENT VARIABLE) 
# WHICH WILL BE USED AS AN INPPUT TO ALL REGRESSION MODELS

# use columns names from the dataframe created previously to select predictor variables for the model 
X = df.iloc[:, 1:5].values #as predictors we are going to use following values: "NDVI_mean", "NDI45_mean", "SAVI_mean" and "NDMI_mean"
y = df.iloc[:, 0].values #dependent variable - AGB (tons/ha) is stored in the first column of the dataframe. 

#MULTIPLE LINEAR REGRESSION
#import necessary classes to run multiple linear regression:
from sklearn import datasets
from sklearn import linear_model 
from sklearn.model_selection import train_test_split
import statsmodels.api as sm #display the results of statistics of the model 
#preparation of training and test dataset taking as an input training dataframe prepared previously, which contains all predictors and dependent variable values
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3) #30% of dataset will be left to test dataset in order to validate model
from sklearn.linear_model import LinearRegression # Import linear regression class 
mlr_regressor = LinearRegression()   #perform multiple linear regression 
mlr_regressor.fit(x_train,y_train)   #fit training dataframe to the model prepared
y_pred= mlr_regressor.predict(x_test) #predict new values of dependent variable (AGB values in our case), using test data remained
#WRITE CALCULATION OF THE RESULTS TO SEPARATE DATAFRAME AND DISPLAY :
mlr_results = {'Intercept':  [mlr_regressor.intercept_],
        'Coefficients': [mlr_regressor.coef_],
        'r-squared score': [mlr_regressor.score(x_train,y_train)]}
mlr_df = pd.DataFrame (mlr_results, columns = ['Intercept','Coefficients','r-squared score'])
mlr_df

In [ ]:
# random forest model

#Preapare training dataset for the model 
features = df.iloc[:, 1:5].values #predictor variables 
AGB_obs = df.iloc[:, 0].values #defined dependent variable
from sklearn.model_selection import train_test_split 
#import class which enables to create the training and test data based on the input parameters 
train_features, test_features, train_AGB_obs, test_AGB_obs = train_test_split(features, AGB_obs, test_size = 0.3, random_state=0) 
#split training dataset into train and test data in the proportion that 30% of the dataset is saved for testing

#RUN RANDOM FOREST REGRESSION MODEL
from sklearn.ensemble import RandomForestRegressor #Import RandomForestRegressor class from sklearn module
RFReg = RandomForestRegressor(n_estimators= 100, max_depth = 3, n_jobs=-1, random_state= 0) # create Random Forest model using 500 as a number of estimators
RFReg.fit(train_features, train_AGB_obs) #fit the training dataset into the model 

#predict AGB values based on the created model with parameters pre-defined
AGB_obs_pred_rf = RFReg.predict((test_features))
from sklearn import metrics #assessing the results of estimation using R2 metric 
r_square_rf = metrics.r2_score(test_AGB_obs, AGB_obs_pred_rf)
print('r-square:', r_square_rf) #dipslay r2 metric of the calculated model 


#here we will produce the numerical values of the importance of each VIs value to the final model 
rf_features = df.iloc[:, 1:5] #list of predictors used in the model 
features_list = list(rf_features.columns) # list the features used in the model
importances = list(RFReg.feature_importances_) # importance of each predictor on the whole model 
#the values of the importance will be calculated based on the model 
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(features_list, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];#display results as a pair observations



In [ ]:
# estimate above ground biomass based on linear regression model

#Here we create the geotiff file which will contain predicted AGB values 
inpRaster= '/shared/Training/PY02_ForestBiomass_Sentinel2/Processing/VIs_stack.tif' 
#define path to the input raster (on which we will make prediction
df_train = pd.read_csv('training_dataset.csv') #path to the dataset with the training data 
data = df_train[['NDVI','NDI45','SAVI','NDMI']] # list of predictors 
label = df_train['AGB_2017'] # predicted variable (dependent variable AGB)
ds = gdal.Open(inpRaster, gdal.GA_ReadOnly) #with gdal module we will open the input raster 
rows = ds.RasterYSize #we need to asisgn the size of the raster - rows number in the raster dataset 
cols = ds.RasterXSize # we need to asisgn the size of the raster - columns number in the raster dataset 
bands = ds.RasterCount #we will also get the number of bands in the input raster (in our case 4)
geo_transform = ds.GetGeoTransform() #with geotransform class we will set projection to the raster 
projection = ds.GetProjectionRef() #projection of the raster needs to be the same as input raster 
array = ds.ReadAsArray() #raster will be converted to an array
ds = None
array = np.stack(array,axis=2) #with numpy stack we will join the sequence of the arrays along a new axes  
array = np.reshape(array, [rows*cols,bands]) 
#we give a final shape to the output raster: number of columns and rows and bands must be the same as input file 
test = pd.DataFrame(array, dtype='float32')  # we will create raster with the same datatype 'float32' as input raster 
outRaster = '/shared/Training/PY02_ForestBiomass_Sentinel2/Processing/AGB_mlr.tif' #path to the output raster - where it will be saved
#PREDICTION OF ABOVEGROUND BIOMASS VALUES BASED ON THE TRAINING DATASET AND THE RASTER STACK PROVIDED 
AGB_mlr = mlr_regressor.predict(test) #run the prediction on multiple linear regression model previously created 
estimation = AGB_mlr.reshape((rows,cols)) #reshape output array into the array with the same defined previously number of columns and rows. 

#export classified image using 'createGeotiff' function provided in the section number 3
createGeotiff(outRaster,estimation,geo_transform,projection)

In [ ]:
# agb based on random forest regression model 

#Here we will run script which is very similar to the cell above, this time taking as a predictor of values Random Forest Regression model 
inpRaster= '/shared/Training/PY02_ForestBiomass_Sentinel2/Processing/VIs_stack.tif' 
#define path to the input raster (on which we will make prediction
outRaster = '/shared/Training/PY02_ForestBiomass_Sentinel2/Processing/AGB_RFReg.tif' #path to the output raster - where it will be saved
df_train = pd.read_csv('training_dataset.csv') #path to the dataset with the training data 
data = df_train[['NDVI','NDI45','SAVI','NDMI']] # list of predictors 
label = df_train['AGB_2017']  # predicted variable (dependent variable AGB)
ds = gdal.Open(inpRaster, gdal.GA_ReadOnly) #with gdal module we will open the input raster 
rows = ds.RasterYSize #we need to asisgn the size of the raster - rows number in the raster dataset 
cols = ds.RasterXSize # we need to asisgn the size of the raster - columns number in the raster dataset 
bands = ds.RasterCount  #we will also get the number of bands in the input raster (in our case 4)
geo_transform = ds.GetGeoTransform() #with geotransform class we will set projection to the raster 
projection = ds.GetProjectionRef()  #projection of the raster needs to be the same as input raster 
array = ds.ReadAsArray() #raster will be converted to an array
ds = None
array = np.stack(array,axis=2)  #with numpy stack we will join the sequence of the arrays along a new axes 
array = np.reshape(array, [rows*cols,bands])  
#we give a final shape to the output raster: number of columns and rows and bands must be the same as input file 
test1 = pd.DataFrame(array, dtype='float32') # we will create raster with the same datatype 'float32' as input raster 
#PREDICTION OF ABOVEGROUND BIOMASS VALUES BASED ON THE TRAINING DATASET AND THE RASTER STACK PROVIDED 
AGB_RFReg = RFReg.predict(test1) #run the prediction on Random Forest Regression model previously created 
estimation = AGB_RFReg.reshape((rows,cols)) #reshape output array into the array with the same defined previously number of columns and rows
##export classified image using 'createGeotiff' function provided in the section number 3
createGeotiff(outRaster,estimation,geo_transform,projection)

In [ ]:
# visualizations

#read the output raster and open it
AGB_mlr = rasterio.open('/shared/Training/PY02_ForestBiomass_Sentinel2/Processing/AGB_mlr.tif')
AGB_mlr_array = AGB_mlr.read() # Read all bands as an array
# Calculate statistics for the image
stats = []
for band in AGB_mlr_array:
    stats.append({
    'min': band.min(),
    'mean': band.mean(),
    'median': np.median(band),
    'max': band.max()})
print(stats)

In [ ]:
#read the output raster and open it
AGB_RF = rasterio.open('/shared/Training/PY02_ForestBiomass_Sentinel2/Processing/AGB_RFReg.tif')
AGB_RF_array = AGB_RF.read() # Read all bands as an array
# Calculate statistics for the image
stats = []
for band in AGB_RF_array:
    stats.append({
    'min': band.min(),
    'mean': band.mean(),
    'median': np.median(band),
    'max': band.max()})
print(stats)

In [ ]:
#Plot the reference data (AGB refernce map for the year 2017 and confront it with the results produced with the regression models 
AGB_ref = rasterio.open('/shared/Training/PY02_ForestBiomass_Sentinel2/AuxData/AGB_2017_resampled.tif')
AGB_ref_array = AGB_ref.read() # Read all bands as an array
fig, (ax1, ax2, ax3)= plt.subplots(1,3, figsize=(18,18)) #create a plot of three images one next to another  
ax1.imshow(AGB_mlr_array[0], cmap= 'Greens', vmin = 0, vmax = 400)  #show the first image - AGB derived from the Multiple Linear Regression model
ax1.set_title('AGB [tons/ha] - MLR model') 
ax2.imshow(AGB_RF_array[0], cmap= 'Greens',vmin = 0, vmax = 400) 
ax2.set_title('AGB [tons/ha] - RFR model') 
ax3.imshow(AGB_ref_array[0], cmap= 'Greens',vmin = 0, vmax = 400) 
ax3.set_title('AGB [tons/ha] - reference (BCCI)')
for ax in fig.get_axes(): 
    ax.label_outer() 